In [2]:
import os
from random import random
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim

root_path = 'data/transcriptions'
data_path = os.path.join(root_path,'train.csv')


In [3]:
class Vocab:
    def __init__(self, counter,for_encoder=False, min_freq=None):
        self.sos = "<sos>"
        self.eos = "<eos>"
        self.pad = "<pad>"
        self.unk = "<unk>"
        
        self.pad_idx = 0
        self.unk_idx = 1
        self.sos_idx = 2
        self.eos_idx = 3
        
        if for_encoder:
            self._token2idx = {
                self.pad:self.pad_idx,
                self.unk:self.unk_idx,
            }
        else:
            self._token2idx = {
                self.sos: self.sos_idx,
                self.eos: self.eos_idx,
                self.pad: self.pad_idx,
                self.unk: self.unk_idx,
            }
        self._idx2token = {idx:token for token, idx in self._token2idx.items()}
        
        
        idx = len(self._token2idx)
        min_freq = 0 if min_freq is None else min_freq
        
        for token, count in counter.items():
            if count > min_freq:
                self._token2idx[token] = idx
                self._idx2token[idx]   = token
                idx += 1
        
        self.vocab_size = len(self._token2idx)
        self.tokens     = list(self._token2idx.keys())
    
    def token2idx(self, token):
        return self._token2idx.get(token, self.pad_idx)
    
    def idx2token(self, idx):
        return self._idx2token.get(idx, self.pad)
    
    def sent2idx(self, sent):
        return [self.token2idx(i) for i in sent]
    
    def idx2sent(self, idx):
        return [self.idx2token(i) for i in idx]
    
    def __len__(self):
        return len(self._token2idx)
    
    def __repr__(self):
        
        return '{}'.format(self._token2idx)

In [4]:
class CharactersDataset(Dataset):
    
    def __init__(self,csv_file_path,transform = None):
        self.file = pd.read_csv(csv_file_path,'r')
        self.transform = transform
        self.data = []
        self.characters_vocab = None
        self.transcripts_vocab = None
        self.non_needed_symbols = '\'#$?\\_({)}-:\";!%.1234567890'
        
        self.make_dataset()
       
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        
        x = self.data[idx]['x']
        y = self.data[idx]['y']
        data = {'x':x,'y':y}
        if self.transform:
            data = self.transform(data)
    
        return data
    
    def make_dataset(self):
        characters = set()
        transcripts = set()
        
        for idx in range(len(self.file)):
            item = str(self.file.iloc[idx][0]).split(',')
            
            x = item[1].strip()
            for symbol in self.non_needed_symbols:
                x = x.replace(symbol,'')
            if x == '':
                continue
            y = item[2].replace(' ','')
            self.data.append({'x':x,'y':y})
            for character in x:
                characters.add(character)
            for transcript in y:
                transcripts.add(transcript)
        
        self.characters_vocab = Vocab({v:k for k,v in dict(enumerate(characters,start=2)).items()},for_encoder=True)
        self.transcripts_vocab = Vocab({v:k for k,v in dict(enumerate(transcripts,start=4)).items()})
    
            
    def collate_fn(self, batch): 
        x_values     =  []
        x_lengths    =  []
        y_values_in  =  []
        y_values_out =  []
        
        for item in batch:
            
            x_values.append([self.characters_vocab.token2idx(ch) for ch in item['x']])
            y_values_in.append([self.transcripts_vocab.token2idx(tr) for tr in item['y']])
            y_values_out.append([self.transcripts_vocab.token2idx(tr) for tr in item['y']])
            
        sorted_tuples = sorted(zip(x_values,y_values_in,y_values_out),key=lambda x:len(x[0]),reverse=True)
        x_values     =  [l[0] for l in sorted_tuples]
        y_values_in  =  [l[1] for l in sorted_tuples]
        y_values_out =  [l[2] for l in sorted_tuples]
        
        max_x = len(x_values[0])
        max_y = max(len(l) for l in y_values_in)
            
        for word_index in range(len(x_values)):
            length_of_current_x = len(x_values[word_index])
            length_of_current_y = len(y_values_in[word_index])

            x_lengths.append(length_of_current_x)
            
            for _ in range(1 +  max_x - length_of_current_x):
                x_values[word_index].append(0)
            for _ in range(1+ max_y - length_of_current_y):
                y_values_in[word_index].append(0)
            
            y_values_in[word_index].insert(0,2)
            
        y_values_in_tensor = torch.tensor(y_values_in)
        
        y_values_out = y_values_in        
        for arr_index in range(len(y_values_out)):
            index_of_first_zero = y_values_out[arr_index].index(0)
            y_values_out[arr_index][index_of_first_zero] = 3
            y_values_out[arr_index] = y_values_out[arr_index][1:]+[0]

        x_values_tensor     = torch.tensor(x_values)
        y_values_out_tensor = torch.tensor(y_values_out)
        
        return x_values_tensor,x_lengths,y_values_in_tensor,y_values_out_tensor

In [5]:
dataset = CharactersDataset(data_path)
dataloader = DataLoader(dataset,batch_size=32,shuffle=True,num_workers=2,collate_fn = dataset.collate_fn)

In [6]:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, 
                                           sampler=train_sampler,collate_fn = dataset.collate_fn)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=32,
                                                sampler=valid_sampler,collate_fn = dataset.collate_fn)

In [7]:
class EncoderLSTM(nn.Module):
    def __init__(self,embed_dim,hidden_size,output_size,n_layers = 1,dropout=0):
        super(EncoderLSTM,self).__init__()
        
        self.embedding = nn.Embedding(output_size,embed_dim,padding_idx = 0)
        self.LSTM = nn.LSTM(embed_dim,hidden_size,n_layers,dropout=(0 if n_layers == 1 else dropout),batch_first=True)
    
    def forward(self,input_seq,input_lengths,hidden=None):
        
        embedded = self.embedding(input_seq)
        
        packed = nn.utils.rnn.pack_padded_sequence(embedded,input_lengths,batch_first=True)
        
        outputs,hidden = self.LSTM(packed)
        
        outputs,_ = nn.utils.rnn.pad_packed_sequence(outputs,batch_first=True)
        return outputs,hidden

In [8]:
class AttentionDecoderLSTM(nn.Module):
    def __init__(self,embed_dim,hidden_size,output_size,n_layers=1,dropout=0):
        super(AttentionDecoderLSTM,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        #layers
        self.embedding = nn.Embedding(output_size,embed_dim, padding_idx = 0)
        self.LSTM = nn.LSTM(embed_dim,hidden_size,n_layers,dropout = (0 if n_layers == 1 else self.dropout),batch_first=True)
        self.concat = nn.Linear(hidden_size*2,hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
        
        
    def forward(self,input_step,last_hidden,encoder_outputs):
        
        #input_step (batch_size,seq_len)
        embedded = self.embedding(input_step)
        #embedded(batch_size,seq_len,hidden_dim)
        
        decoder_outputs,hidden = self.LSTM(embedded,last_hidden)
        #(batch_size,decoder_seq_len,hidden_dim)
        #seq_len = 1 if we using teacher forcing
        
        decoder_outputs = decoder_outputs.transpose(1,2)
        #(batch_size,hidden_dim,decoder_seq_len)
        
        attention_weights = encoder_outputs.bmm(decoder_outputs)
        #(batch_size,encoder_seq_len,decoder_seq_len)
        
        attention_weights = attention_weights.transpose(1,2)
        #(batch_size,decoder_seq_len,encoder_seq_len)
        attention_weights = F.softmax(attention_weights,dim=1)
        
        context_vector = attention_weights.bmm(encoder_outputs)
        #(batch_size,decoder_seq_len,hidden_dim)
        
        decoder_outputs = decoder_outputs.transpose(1,2)
        #(batch_size,decoder_seq_len,hidden_dim)
        
        concated = torch.cat((decoder_outputs,context_vector),2)
        concated_output = torch.tanh(self.concat(concated))
        
        prediction = self.out(concated_output)
        #prediction(seq_len,batch_size,output_dim) if no teacher_forcing
        
        return prediction,hidden

In [9]:
device = torch.device("cpu")
BATCH_SIZE        = 32
HIDDEN_DIM        = 64
INPUT_VOCAB_SIZE  = len(dataset.characters_vocab)
OUTPUT_VOCAB_SIZE = len(dataset.transcripts_vocab)
encoder = EncoderLSTM(BATCH_SIZE,HIDDEN_DIM,INPUT_VOCAB_SIZE).to(device)
decoder = AttentionDecoderLSTM(BATCH_SIZE,HIDDEN_DIM,OUTPUT_VOCAB_SIZE).to(device)

In [11]:
for batch in dataloader:
    kek = batch
    break
x = kek[0].to(device)
x_lengths = kek[1]
y_in = kek[2].to(device)

In [13]:
encoder_outputs,encoder_hidden = encoder(x,x_lengths)
decoder_outputs = decoder(y_in,encoder_hidden)
at = decoder_outputs.bmm(encoder_outputs.transpose(1,2))

In [ ]:
at2 = torch.sum(decoder_outputs*encoder_outputs,dim=2)

In [9]:
import random
class seq2seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    
    def forward(self,x,x_lengths,y,teacher_forcing_ratio = 0.3):
        
        encoder_outputs,encoder_hidden = self.encoder(x,x_lengths)
        predictions,hidden = self.decoder(y,encoder_hidden,encoder_outputs)
        
       
        #hidden = self.encoder(x)
        #input_token = y[:,0]
        
        #for t in range(1,seq_len):
         #   output,hidden = self.decoder(input_token,hidden)
          #  outputs[t] = output
           # teacher_force = random.random() < teacher_forcing_ratio
            #top1 = output.max(1)[1]
           # input_token = (y[:,t] if teacher_force else top1)
        
        return predictions
    
    def predict(self,x,x_lengths):
        
        #batch_size = 1!
        x.unsqueeze_(0)
        encoder_outputs,hidden = self.encoder(x,x_lengths)
        char_to_input = torch.LongTensor([[2]])
        preds = torch.tensor(char_to_input).to(self.device)
        
        while True:
            predictions,hidden = self.decoder(preds, hidden,encoder_outputs)
            index_of_next = torch.argmax(predictions)
            our_value = index_of_next.item()

            if our_value == 3:
                break
            char_to_input = index_of_next.unsqueeze_(0).unsqueeze_(0)
            preds = torch.cat((preds,char_to_input),1)
        return preds

In [ ]:
model = seq2seq(encoder,decoder,device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [ ]:
def train(model, iterator, optimizer, criterion, clip,epoch,train_loss_list):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        x = batch[0].to(device)
        x_lengths = batch[1]
        y_in = batch[2].to(device)
        y_out = batch[3].to(device)
        
        optimizer.zero_grad()
        
        output = model(x,x_lengths, y_in)
        #output dim (y_seq_len,batch_size,output_dim)
        output = output.view(output.shape[0]*output.shape[1],-1)
        y_out = y_out.view(-1)
        
        loss = criterion(output, y_out)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        train_loss_list.append(loss.item())

        if i % 500 == 0:
            plot(epoch, i, train_loss_list)
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            x = batch[0].to(device)
            x_lengths = batch[1]
            y_in = batch[2].to(device)
            y_out = batch[3].to(device)
            
            output = model(x,x_lengths,y_in,0) #turn off teacher forcing
            
            output = output.view(output.shape[0]*output.shape[1],-1)
            y_out = y_out.view(-1)

            loss = criterion(output, y_out)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)



In [ ]:
x[0].shape

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
def plot(epoch, step, train_losses):
    clear_output(True)
    plt.title(f'Epochs {epoch}, step {step}')
    plt.plot(train_losses)
    plt.show()

In [ ]:
import math

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
train_losses = []
for epoch in range(N_EPOCHS):
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP,epoch,train_losses)
    valid_loss = evaluate(model, validation_loader, criterion)
    
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

In [ ]:
torch.save(encoder.state_dict(),'encoder_weights')
torch.save(decoder.state_dict(),'decoder_weights')
torch.save(model.state_dict(),'model_weights')

In [10]:
encoder.load_state_dict(torch.load('encoder_weights'))
decoder.load_state_dict(torch.load('decoder_weights'))
model = seq2seq(encoder,decoder,device)
model.load_state_dict(torch.load('model_weights'))

In [18]:
for batch in validation_loader:
    kek = batch
    break


In [32]:
kek[0][31]

tensor([10, 24, 10, 18, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0])

In [21]:
def predict_batch(model,batch,batch_size):
    x_tokens = []
    y_tokens_predicted = []
    y_tokens_true = []
    for i in range(batch_size):
        print(i)
        x_tokens.append(''.join([dataset.characters_vocab.idx2token(k.item()) for k in batch[0][i] if k.item() != 0]))
        y_pred = model.predict(batch[0][i].to(device),[batch[1][i]])
        y_tokens_predicted.append(''.join([dataset.transcripts_vocab.idx2token(k) for k in y_pred]))
        y_tokens_true.append(''.join([dataset.transcripts_vocab.idx2token(k.item()) for k in batch[2][i] if k.item() != 0 and k.item() != 2]))
    for i in range(len(x_tokens)):
        print('Input:',x_tokens[i])
        print('Predicted_output:',y_tokens_predicted[i])
        print('True output:',y_tokens_true[i],'\n\n')

In [ ]:
inp = ''
out = ''
for i in x: 
    inp += dataset.characters_vocab.idx2token(i.item())
for i in y:
    out += dataset.transcripts_vocab.idx2token(i.item())
inp,out

In [22]:
predict_batch(model,kek,32)

0


/home/yerlan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: index out of range at /pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:191

In [ ]:
a = torch.argmax(predictions).unsqueeze_(0).unsqueeze_(0).shape

In [ ]:
kek[0][1]

In [ ]:
yt.shape

In [ ]:
x_tokens

In [ ]:
predict_batch(model,kek,32)

In [ ]:
predict_batch(model,kek,32)